In [1]:
import pandas as pd


In [19]:
candidates_df=pd.read_csv('data/candidates.csv')
candidates_df.head()


,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-56.08,-67.85,-311.92,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,53.21,-244.41,-245.17,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.66,-121.80,-286.62,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-33.66,-72.75,-308.41,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-32.25,-85.36,-362.51,0


In [22]:
candidates_df['class'].value_counts()


class
0    549714
1      1351
Name: count, dtype: int64

In [4]:
annotations_df=pd.read_csv('data/annotations.csv')
annotations_df.head()

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150


In [23]:
annotations_df['seriesuid'].value_counts()

seriesuid
1.3.6.1.4.1.14519.5.2.1.6279.6001.176030616406569931557298712518    12
1.3.6.1.4.1.14519.5.2.1.6279.6001.195557219224169985110295082004     9
1.3.6.1.4.1.14519.5.2.1.6279.6001.219428004988664846407984058588     9
1.3.6.1.4.1.14519.5.2.1.6279.6001.202187810895588720702176009630     9
1.3.6.1.4.1.14519.5.2.1.6279.6001.328789598898469177563438457842     9
                                                                    ..
1.3.6.1.4.1.14519.5.2.1.6279.6001.910607280658963002048724648683     1
1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674661221381920536987     1
1.3.6.1.4.1.14519.5.2.1.6279.6001.110678335949765929063942738609     1
1.3.6.1.4.1.14519.5.2.1.6279.6001.108231420525711026834210228428     1
1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492     1
Name: count, Length: 601, dtype: int64

In [6]:
import logging
import logging.handlers

from sympy import root

root_logger=logging.getLogger()
root_logger.setLevel(logging.INFO)

for handler in list(root_logger.handlers):
    root_logger.removeHandler(handler)

logfmt_str="%(asctime)s %(levelname)-8s pid:%(process)d %(name)s:%(lineno)03d:%(funcName)s %(message)s"

formatter=logging.Formatter(logfmt_str)
streamHandler=logging.StreamHandler()
streamHandler.setFormatter(formatter)
streamHandler.setLevel(logging.DEBUG)

root_logger.addHandler(streamHandler)


In [ ]:
import copy
import csv
import functools
import glob
import os

from collections import namedtuple

import SimpleITK as sitk
import numpy as np

import torch
import torch.cuda
from torch.utils.data import Dataset

#from util.disk import getCache
#from util.util import XyzTuple, xyz2irc
#from util.logconf import logging

log=logging.getLogger(__name__)
log.setLevel(logging.DEBUG)


CandidateInfoTuple=namedtuple(
    'CandidateinfoTuple',
    'isNodule_bool,diameter_mm,series_uid,center_xyz',
)

@functools.lru_cache(1)
def getCandidateInfoList(requireOnDisk_bool=True):
    mhd_list=glob.glob('data/subset0/*.mhd')
    presentOnDisk_set={os.path.split(p)[-1][:-4] for p in mhd_list}

    diameter_dict={}
    with open('data/annotations.csv',"r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid=row[0]
            annotationCenter_xyz=tuple([float(x) for x in row[1:4]])
            annotationDiameter_mm=float(row[4])

            diameter_dict.setdefault(series_uid,[]).append(
                (annotationCenter_xyz, annotationDiameter_mm)
            )
    
    candidateInfo_list=[]
    with open('data/candidates.csv',"r") as f:
        for row in list(csv.reader(f))[1:]:
            series_uid=row[0]

            if series_uid not in presentOnDisk_set and requireOnDisk_bool:
                continue
            isNodule_bool=bool(int(row[4]))
            candidateCenter_xyz=tuple([float(x) for x in row[1:4]])

            candidateDiameter_mm=0.0
            for annotation_tup in diameter_dict.get(series_uid,[]):
                annotationCenter_xyz,annotationDiameter_mm=annotation_tup

                for i in range(3):
                    delta_mm=abs(candidateCenter_xyz[i]-annotationCenter_xyz[i])
                    if delta_mm>annotationDiameter_mm/4:
                        break
                else:
                    candidateDiameter_mm=annotationDiameter_mm
                    break
            candidateInfo_list.append(CandidateInfoTuple(
                isNodule_bool,
                candidateDiameter_mm,
                series_uid,
                candidateCenter_xyz
            ))
    candidateInfo_list.sort(reverse=True)
    return candidateInfo_list

               





In [8]:
#util
import collections
IrcTuple=collections.namedtuple('IrcTuple', ['index','row','column'])
XyzTuple=collections.namedtuple('XyzTuple', ['x','y','z'])
from util.util import xyz2irc



In [9]:
class Ct:
    def __init__(self, series_uid):
        mhd_path=glob.glob(
            'data/subset0/{}.mhd'.format(series_uid)
        )[0]

        ct_mhd=sitk.ReadImage(mhd_path)
        ct_a=np.array(sitk.GetArrayFromImage(ct_mhd))
        ct_a.clip(-1000, 1000,ct_a)

        self.series_uid=series_uid
        self.hu_a=ct_a

        self.origin_xyz=XyzTuple(*ct_mhd.GetOrigin())
        self.vxSize_xyz=XyzTuple(*ct_mhd.GetSpacing())
        self.direction_a=np.array(ct_mhd.GetDirection()).reshape(3,3)

    def getRawCandidate(self,center_xyz,width_irc):
        center_irc=xyz2irc(
            center_xyz,
            self.origin_xyz,
            self.vxSize_xyz,
            self.direction_a,
        )

        slice_list=[]
        for axis,center_val in enumerate(center_irc):
            start_ndx=int(round(center_val-width_irc[axis]/2))
            end_ndx=int(start_ndx+width_irc[axis])

            assert center_val>=0 and center_val<self.hu_a.shape[axis], repr([self.series_uid, center_xyz,self.origin_xyz,self.vxSize_xyz,center_irc,axis])

            if start_ndx<0:
                start_ndx=0
                end_ndx=int(width_irc[axis])

            if end_ndx>self.hu_a.shape[axis]:
                end_ndx=self.hu_a.shape[axis]
                start_ndx=int(self.hu_a.shape[axis]-width_irc[axis])

            slice_list.append(slice(start_ndx,end_ndx))
        
        ct_chunk=self.hu_a[tuple(slice_list)]

        return ct_chunk,center_irc


In [10]:




def getCt(series_uid):
    return Ct(series_uid)


def getRawCandidate(series_uid,center_xyz,width_irc):
    ct=getCt(series_uid)
    ct_chunk,center_irc=ct.getRawCandidate(center_xyz,width_irc)
    return ct_chunk, center_irc
    

In [11]:
class LunaDataset(Dataset):
    def __init__(self, val_stride=0, isValSet_bool=None,series_uid=None):
        self.candidateInfo_list=copy.copy(getCandidateInfoList())

        if series_uid:
            self.candidateInfo_list=[
                x for x in self.candidateInfo_list if x.series_uid==series_uid
            ]
        if isValSet_bool:
            assert val_stride>0, val_stride
            self.candidateInfo_list=self.candidateInfo_list[::val_stride]

            assert self.candidateInfo_list
        elif val_stride>0:
            del self.candidateInfo_list[::val_stride]
            assert self.candidateInfo_list

        log.info("{!r}: {} {} samples".format(
            self,
            len(self.candidateInfo_list),
            "validation" if isValSet_bool else "training",
        ))

    def __len__(self):
        return len(self.candidateInfo_list)
    
    def __getitem__(self, ndx):
        candidateInfo_tup=self.candidateInfo_list[ndx]
        width_irc=(32,48,48)

        candidate_a, center_irc=getRawCandidate(
            candidateInfo_tup.series_uid,
            candidateInfo_tup.center_xyz,
            width_irc
        )
        candidate_t=torch.from_numpy(candidate_a)
        candidate_t=candidate_t.to(torch.float32)
        candidate_t=candidate_t.unsqueeze(0)

        pos_t=torch.tensor([
            not candidateInfo_tup.isNodule_bool,
            candidateInfo_tup.isNodule_bool,
        ],
        dtype=torch.long,
        )

        return (
            candidate_t,
            pos_t,
            candidateInfo_tup.series_uid,
            torch.tensor(center_irc)
        )

In [12]:
LunaDataset(val_stride=10, isValSet_bool=True)


FileNotFoundError: [Errno 2] No such file or directory: 'data/annotattions.csv'